В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: standart scale

Добавление шума: только к категориальным признакам OHE

In [18]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [2]:
dataname = 'adult_ON_SC'
model_short = 'tabddpm_ON_SC'

In [3]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [4]:
model_save_path_hist = []
for sigma in [0, 0.001, 0.01, 0.1, 0.25, 0.5]:
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i, sigmas in enumerate(range(10)):
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_mult_{i}"
    model_save_path_hist.append(model_save_path)

### Sample + Eval

In [5]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_SC',
 'method': 'tabddpm_ON_SC_const_0',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/adult_ON_SC/tabddpm_ON_SC_const_0.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_SC/initial_real.csv',
 'test_path': './synthetic/adult_ON_SC/initial_test.csv',
 'info_path': './data/adult_ON_SC/initial_info.json',
 'save_path': './synthetic/adult_ON_QnSC/initial_tabddpm_test.csv'}

In [21]:
overall_metrics = {}

for model_save_path in model_save_path_hist[:1]:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise(CONFIG, model_save_path=model_save_path, sigmas=None,
                                         dataname=dataname, device=device)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv')

    # подсчет метрик
    # overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') - ошибка
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

In [22]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':'TabDDPM ON', 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [24]:
final_metrics_table.round(3).sort_values(('', 'Type'))

similarity  \
        Model                       Type         Data Column Shapes Score, %   
0  TabDDPM ON      tabddpm_ON_SC_const_0  adult_ON_SC                 89.149   
0  TabDDPM ON  tabddpm_ON_SC_const_0_001  adult_ON_SC                 89.096   
0  TabDDPM ON   tabddpm_ON_SC_const_0_01  adult_ON_SC                 30.047   
0  TabDDPM ON    tabddpm_ON_SC_const_0_1  adult_ON_SC                 63.131   
0  TabDDPM ON   tabddpm_ON_SC_const_0_25  adult_ON_SC                 63.050   
0  TabDDPM ON    tabddpm_ON_SC_const_0_5  adult_ON_SC                 50.832   
0  TabDDPM ON       tabddpm_ON_SC_mult_0  adult_ON_SC                 67.627   
0  TabDDPM ON       tabddpm_ON_SC_mult_1  adult_ON_SC                 47.500   
0  TabDDPM ON       tabddpm_ON_SC_mult_2  adult_ON_SC                 66.350   
0  TabDDPM ON       tabddpm_ON_SC_mult_3  adult_ON_SC                 57.031   
0  TabDDPM ON       tabddpm_ON_SC_mult_4  adult_ON_SC                 51.920   
0  TabDDPM ON       tabddpm_ON_SC_mult_5  adult_ON_SC                 52.497   
0  TabDDPM ON       tabddpm_ON_SC_mult_6  adult_ON_SC                 47.946   
0  TabDDPM ON       tabddpm_ON_SC_mult_7  adult_ON_SC                 63.180   
0  TabDDPM ON       tabddpm_ON_SC_mult_8  adult_ON_SC                 57.870   
0  TabDDPM ON       tabddpm_ON_SC_mult_9  adult_ON_SC                 65.788   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                      52.828                     70.988   
0                      52.392                     70.744   
0                      26.883                     28.465   
0                      45.859                     54.495   
0                      45.642                     54.346   
0                      48.522                     49.677   
0                      45.534                     56.580   
0                      44.998                     46.249   
0                      47.484                     56.917   
0                      41.458                     49.245   
0                      48.901                     50.411   
0                      43.645                     48.071   
0                      35.678                     41.812   
0                      41.788                     52.484   
0                      32.350                     45.110   
0                      31.877                     48.833   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              10.851   
0                                              10.904   
0                                              69.953   
0                                              36.869   
0                                              36.950   
0                                              49.168   
0                                              32.373   
0                                              52.500   
0                                              33.650   
0                                              42.969   
0                                              48.080   
0                                              47.503   
0                                              52.054   
0                                              36.820   
0                                              42.130   
0                                              34.212   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  16.302   
0                                                  16.445   
0                                                  25.964   
0                                                  18.228   
0                                                  18.033   
0                                                  31.896   
0                            

In [27]:
import os
directory = f'./eval/total/{dataname}'
if not os.path.exists(directory):
    os.makedirs(directory)

In [28]:
final_metrics_table.sort_values(('', 'Type')).to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)